In [1]:
from models.model import resnet_feature_extraction, model_init
from torchviz import make_dot
import torch
# import matplotlib.pyplot as plt
import copy

In [2]:
extractor = resnet_feature_extraction(101, 2, 0.02)
before_state_dict = copy.deepcopy(extractor.state_dict())

## Debugging print statements

## Print model parameters:
# for param in segmenter.parameters():
#   print(param.data)
## Print parameters by accesing the state_dict:
# print("Initial weights of the first layer: ", before_state_dict)

model_init(extractor,101,2)
after_state_dict = extractor.state_dict()
## Print model parameters after loading the pretrained weights:
# for param in pretrained_segmenter.parameters():
#   print(param.data)
## Print parameters of the updated model by accesing the state_dict:
# print("Weights of the first layer after loading pretrained weights: ", after_state_dict)

# Compare the state_dicts
for layer in before_state_dict:
    # print("Checking layer " + layer + " ...")
    if not torch.allclose(before_state_dict[layer], after_state_dict[layer]):
        print(f'{layer} has been updated')

## Print the name of every single layer in the model
# for name, module in segmenter.named_children():
    # print(name, module)

conv1.module.weight has been updated
bn1.bn_0.weight has been updated
bn1.bn_0.bias has been updated
bn1.bn_0.running_mean has been updated
bn1.bn_0.running_var has been updated
bn1.bn_1.weight has been updated
bn1.bn_1.bias has been updated
bn1.bn_1.running_mean has been updated
bn1.bn_1.running_var has been updated
layer1.0.conv1.module.weight has been updated
layer1.0.bn1.bn_0.weight has been updated
layer1.0.bn1.bn_0.bias has been updated
layer1.0.bn1.bn_0.running_mean has been updated
layer1.0.bn1.bn_0.running_var has been updated
layer1.0.bn1.bn_1.weight has been updated
layer1.0.bn1.bn_1.bias has been updated
layer1.0.bn1.bn_1.running_mean has been updated
layer1.0.bn1.bn_1.running_var has been updated
layer1.0.conv2.module.weight has been updated
layer1.0.bn2.bn_0.weight has been updated
layer1.0.bn2.bn_0.bias has been updated
layer1.0.bn2.bn_0.running_mean has been updated
layer1.0.bn2.bn_0.running_var has been updated
layer1.0.bn2.bn_1.weight has been updated
layer1.0.bn2.bn_

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
color_shape = (1, 3, 320, 320)
random_color_tensor = torch.randn(*color_shape,device=device)
# plt.imshow(random_color_tensor.squeeze().numpy())

In [5]:
depth_shape = (1, 3, 320, 320)

random_depth_tensor = torch.randn(*depth_shape,device=device)
# plt.imshow(random_depth_tensor.squeeze().numpy())

In [6]:
random_color_tensor.device

device(type='cuda', index=0)

In [7]:
extractor.to(device)
# segmenter = torch.nn.DataParallel(segmenter,[0] ,device)

ResNetFeatures(
  (dropout): ModuleParallel(
    (module): Dropout(p=0.5, inplace=False)
  )
  (conv1): ModuleParallel(
    (module): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  )
  (bn1): BatchNorm2dParallel(
    (bn_0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (relu): ModuleParallel(
    (module): ReLU(inplace=True)
  )
  (maxpool): ModuleParallel(
    (module): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): ModuleParallel(
        (module): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (bn1): BatchNorm2dParallel(
        (bn_0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [8]:
inputs = [random_color_tensor,random_depth_tensor]


In [9]:
for parameter in extractor.parameters():
    device = parameter.device
    break

print(device)

cuda:0


In [10]:
out = extractor(inputs)

In [11]:
out[3][0].shape

torch.Size([1, 2048, 10, 10])

In [ ]:
# Pick one feature map for rgb and depth
rgb_features = out[3][0]
depth_features = out[3][1]

In [ ]:
concat_features = torch.cat((rgb_features, depth_features), dim=1)

In [ ]:
# Apply 3x3 convolution
fusion_layer = torch.nn.Conv2d(concat_features.size(1), 2048, kernel_size=3, padding=1,device=device)
fused_features = fusion_layer(concat_features)

In [ ]:
fused_features.shape